In [1]:
#dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

#import api key
from config import weather_api_key

#incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#output file (csv)
output_data_file = "output_data/cities.csv"

#range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)
lat_range = (-90, -18)
lng_range = (-180, -150)

#formatting for bold text
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

In [3]:
#printing out the minneapolis json to see the data
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = f"{url}appid={weather_api_key}&units={units}&q="
response = requests.get(query_url + "minneapolis").json()
response

{'coord': {'lon': -93.26, 'lat': 44.98},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 18.03,
  'feels_like': 7.12,
  'temp_min': 15.01,
  'temp_max': 21,
  'pressure': 1023,
  'humidity': 72},
 'visibility': 16093,
 'wind': {'speed': 9.17, 'deg': 160},
 'clouds': {'all': 1},
 'dt': 1579639504,
 'sys': {'type': 1,
  'id': 5829,
  'country': 'US',
  'sunrise': 1579614231,
  'sunset': 1579647856},
 'timezone': -21600,
 'id': 5037649,
 'name': 'Minneapolis',
 'cod': 200}

In [4]:
#List for holding lat_lngs and cities
lat_lngs = []
cities = []

#create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=-18.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=-150.000, size=1500)
lat_lngs = zip(lats, lngs)

# Add a one second interval between queries to stay within API query limits
time.sleep(1)

#identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

#print the city count to confirm sufficient count
len(cities)

10

In [5]:
#Perform API Calls
#write something more descriptive...

#base url
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
#build partial query url
query_url = f"{url}appid={weather_api_key}&units={units}&q="

city_info = []

#in order to print the print log according to how the homework looks (change comment before final draft)
i = 1
set_number = 1

print("Beginning Data Retrieval") 
print("--------------------------------")

for j, city in enumerate(cities):
    
    if (j % 50 == 0 and j >= 50):
        set_number = set_number + 1
        i = 0
    print(f"Processing Record {i} of Set {set_number} | {city}")
    i=i+1
   
    response = requests.get(query_url + city).json()
    
    # Add a one second interval between queries to stay within API query limits
    time.sleep(1)
    
    try:
        lat = response['coord']['lat']
        name = response['name']
        cloudiness = response['clouds']['all']
        country = response['sys']['country']
        date = response['dt']
        humidity = response['main']['humidity']
        lngs = response['coord']['lon']
        max_temp = response['main']['temp_max']
        wind_speed = response['wind']['speed']
        city_info.append({"City": name,
                          "Cloudiness": cloudiness,
                          "Country": country,
                          "Date": date,
                          "Humidity": humidity,
                          "Lat": lat,
                          "Lng": lngs,
                          "Max Temp": max_temp,
                          "Wind Speed": wind_speed})
    except:
        print("City " + color.BOLD + "not " + color.END + "found. Skipping...")
        pass

print("------------------------")
print("Data Retrieval Complete")
print("------------------------")

Beginning Data Retrieval
--------------------------------
Processing Record 1 of Set 1 | vaini
Processing Record 2 of Set 1 | mataura
Processing Record 3 of Set 1 | alofi
Processing Record 4 of Set 1 | avera
Processing Record 5 of Set 1 | avarua
Processing Record 6 of Set 1 | papara
Processing Record 7 of Set 1 | pangai
Processing Record 8 of Set 1 | neiafu
Processing Record 9 of Set 1 | moerai
Processing Record 10 of Set 1 | tevaitoa
------------------------
Data Retrieval Complete
------------------------


In [6]:
#export the city data into a csv
#display the dataframe

city_df = pd.DataFrame(city_info)

city_df.to_csv('cities.csv', index_label="City_ID")

city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Vaini,40,TO,1579639751,83,-21.20,-175.20,78.8,2.24
1,Mataura,100,NZ,1579639752,96,-46.19,168.86,59.0,1.99
2,Alofi,40,NU,1579639753,69,-19.06,-169.92,80.6,5.82
3,Avera,1,US,1579639754,32,33.19,-82.53,42.8,9.17
4,Avarua,100,CK,1579639755,100,-21.21,-159.78,75.2,3.36


In [ ]:
#plot latitude v temperature